### Goals of today: 

Once you have the library psycopg2, be able to:
* connect to SQL alchemy or pandas dataframes 
* think critically about why splat is not good to use except to explore the data in the beginning 
* dealing with transactions

In [1]:
! pip install psycopg2
# get a library 

    100% |████████████████████████████████| 368kB 1.8MB/s ta 0:00:01
  Running setup.py bdist_wheel for psycopg2 ... done
  Stored in directory: /Users/sherzyang/Library/Caches/pip/wheels/3b/d0/9c/fbbaca1e768e108fdcb88a9a50ea43de141adf842741f8623f
Successfully built psycopg2


In [1]:
import psycopg2

In [92]:
# connects to a database 
#if the company uses redshift this is also the way to connect to the database 

conn = psycopg2.connect("dbname=sherzyang")

In [93]:
#make a cursor 
cur = conn.cursor()

Let's take a look at our database

In [94]:
query = """
SELECT * FROM cats;
"""

In [95]:
cur.execute(query)

In [67]:
cur.fetchall()

[(1, 'Bob', 20, 'Tiger', None, 'Striped', 7),
 (2, 'Sherry', 15, 'Nina', None, 'Black', 7)]

In order to work with for item in cur: print(item), we do not run fetchall

In [34]:
query = """
SELECT * FROM cats
WHERE age < 16;
"""

In [35]:
cur.execute(query)

In [36]:
for item in cur: 
    print(item)

(2, 'Sherry', 15, 'Nina', None, 'Black', 7)


Put the data into a pandas dataframe 

In [96]:
import pandas as pd 

In [97]:
data = cur.fetchall()

In [98]:
pd.DataFrame(data)

,0,1,2,3,4,5,6
0,1,Bob,20,Tiger,NaN,Striped,7
1,2,Sherry,15,Nina,NaN,Black,7
2,3,Miles,8,Tabby,24.0,Orange,10


One way to rename our columns, but it is not the best way 

In [99]:
columns = [col.name for col in cur.description]

In [100]:
pd.DataFrame(data, columns=columns)

,id,name,age,breed,owner_id,color,fluffiness
0,1,Bob,20,Tiger,NaN,Striped,7
1,2,Sherry,15,Nina,NaN,Black,7
2,3,Miles,8,Tabby,24.0,Orange,10


This is the way to name columns, do not call splat, call specific columns 

In [101]:
columns = ['fluffiness','color','owner_id']

In [102]:
query = f"""
SELECT {','.join(columns)}
FROM cats
WHERE age < 16;
"""

In [103]:
print(query)


SELECT fluffiness,color,owner_id
FROM cats
WHERE age < 16;



In [104]:
cur.execute(query)
data = cur.fetchall()
pd.DataFrame(data,columns=columns)

,fluffiness,color,owner_id
0,7,Black,NaN
1,10,Orange,24.0


In [110]:
# to recover from an error 
conn.rollback() # a lighter weight solution to closing and reopening

In [111]:
#say I make a change here, I need to commit them 

In [118]:
query = """
DELETE FROM cats
WHERE id = 3;
"""
cur.execute(query)

In [119]:
conn.commit()